# data set Titanic


## importamos las librerias necesarias para el data set 

In [703]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
# Herramientas de preprocesamiento de Scikit-learn
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

## 1. analisis inicial 

### leemos el archivo csv train.csv


In [704]:
# data set cargado en crudo
df_raw= pd.read_csv('train.csv')

# data set copia para editarlo

df_procesado = df_raw.copy()

### mostramos las dimensiones del data set


In [705]:
df_procesado.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


### df.info()

despues de ejecutar el df.info() obtuvimos la siguiente información 
tenemos 12 columnas las cuales estan dividas en: id del pasajero, sobrevivido, clase en la que estaba (clase siendo referencia a la clase social en la que se estaba alojando el tripulante), que edad tenia, si viajaba con un acompañante o no, el numero de ticket, el puerto de embarque, tarifa, sexo, nombre, cuantos padres o hijos viajaban en la embarcacion.

### ¿cuantos datos nulos tenemos?

In [706]:
df_procesado.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

en las casillas donde se muestran datos vacios son las siguientes: 
1. Age con 177 datos nulos 
2. cabin con 687 datos nulos
3. embarked con 2 datos nulos

### df.head

con df.head() nos muestra los primeros 5 registros del data set 

In [707]:
df_procesado.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 2. estrategia para la columna Age (edad)

se utilizara la mediana por que es la que permite encontrar el valor en el centro y para los datos que tenemos es la mas optima 

In [708]:
# Seleccionas la columna 'Age' y luego calculas su mediana
mediana_edad = df_procesado['Age'].median()

print(f"La mediana de la edad es: {mediana_edad} años ")

 # Es un atajo para df['Age'].median()
# mediana_edad = df.Age.median()

 #print(f"La mediana de la edad es: {mediana_edad}")

La mediana de la edad es: 28.0 años 


#### rellenar los valores faltantes



In [709]:
df_procesado['Age'].fillna(value=mediana_edad, inplace = True)

C:\Users\PC\AppData\Local\Temp\ipykernel_19292\1735000010.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_procesado['Age'].fillna(value=mediana_edad, inplace = True)


In [710]:
df_procesado['Age'].isnull().sum()

np.int64(0)

mostramos nuevamente los datos del dataSet despues de aplicar los cambios de llenar los valores faltantes en la columna Age con la mediana 

In [711]:
df_procesado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## 3. estrategia de la columna cabin (camarote)

ya que la columna camarote puede tener relacion con la columna de la clase social, por lo cual no es muy recomendable borrarla 

lo que se va hacer es crear una nueva columna llamada tiene_camarote donde el valor 1 significa que si hay un dato en el camarote, donde 0 sea igual a que no tiene datos en el camarote si era nulo (null)

In [712]:
df_procesado['tiene_Camarote'] = df_procesado['Cabin'].notnull().astype(int)

In [713]:
df_procesado.info()

df_procesado.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PassengerId     891 non-null    int64  
 1   Survived        891 non-null    int64  
 2   Pclass          891 non-null    int64  
 3   Name            891 non-null    object 
 4   Sex             891 non-null    object 
 5   Age             891 non-null    float64
 6   SibSp           891 non-null    int64  
 7   Parch           891 non-null    int64  
 8   Ticket          891 non-null    object 
 9   Fare            891 non-null    float64
 10  Cabin           204 non-null    object 
 11  Embarked        889 non-null    object 
 12  tiene_Camarote  891 non-null    int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 90.6+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,tiene_Camarote
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


borramos la columna cabin para evitar ruido en la data

In [714]:
df_procesado.drop('Cabin',axis=1, inplace= True)

df_procesado.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,tiene_Camarote
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,0


## 4. estrategia para columna embarked

In [715]:
# saber cuales son los valores faltantes en la columna Embarked llenandola con la moda
moda_Embarked = df_procesado['Embarked'].mode()[0]

print(f" la moda es {moda_Embarked}")



 la moda es S


In [716]:
# llenar los datos faltantes con la moda 
df_procesado['Embarked'].fillna(value= moda_Embarked, inplace= True)


df_procesado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PassengerId     891 non-null    int64  
 1   Survived        891 non-null    int64  
 2   Pclass          891 non-null    int64  
 3   Name            891 non-null    object 
 4   Sex             891 non-null    object 
 5   Age             891 non-null    float64
 6   SibSp           891 non-null    int64  
 7   Parch           891 non-null    int64  
 8   Ticket          891 non-null    object 
 9   Fare            891 non-null    float64
 10  Embarked        891 non-null    object 
 11  tiene_Camarote  891 non-null    int64  
dtypes: float64(2), int64(6), object(4)
memory usage: 83.7+ KB


C:\Users\PC\AppData\Local\Temp\ipykernel_19292\1549376155.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_procesado['Embarked'].fillna(value= moda_Embarked, inplace= True)


creacion de la nueva columna llamada tiene familia 

In [717]:
# --- Celda Nueva: Ingeniería de Características ---

print("Creando la columna 'Tamaño_Familia'...")

# Sumamos las columnas SibSp y Parch, y añadimos 1 (para contar al propio pasajero)
df_procesado['Tamaño_Familia'] = df_procesado['SibSp'] + df_procesado['Parch'] + 1

print("¡Columna 'Tamaño_Familia' creada exitosamente!")

# Verificamos las primeras filas para confirmar que se creó bien
df_procesado[['SibSp', 'Parch', 'Tamaño_Familia']].head()

Creando la columna 'Tamaño_Familia'...
¡Columna 'Tamaño_Familia' creada exitosamente!


,SibSp,Parch,Tamaño_Familia
0,1,0,2
1,1,0,2
2,0,0,1
3,1,0,2
4,0,0,1


# fase 2 preparacion de datos para el analisis 

Objetivo: Ahora que los datos están limpios, debemos "traducirlos" a un formato numérico que los futuros modelos de IA puedan procesar.

### 1.	Codificación de Variables Categóricas: Las columnas Sex, Embarked y la nueva Tiene_Camarote son texto o categorías

o	Acción Requerida: Conviértelas en números. Te recomiendo usar One-Hot Encoding. En tu notebook, explica brevemente por qué esta técnica es preferible a simplemente asignar números como 0, 1, 2.


In [718]:
# guardamos nuestro dataFrame procesado en una nueva variable para esta etapa 
# asi mantenemos el paso anterior guardado por si acaso

df_codificado = df_procesado.copy()

seleccionamos las colummnas a codificar 

In [719]:
columnas_categoricas = ['Sex','Embarked']

usamos pd.get_dummies para aplicar One_hot Encoding

drop_first = True es una buena practica para evitar redundancia de datos

In [720]:
df_codificado = pd.get_dummies(df_codificado, columns=columnas_categoricas, drop_first=True)

convertir solo las nuevas columnas de tipo booleano (true/false) a enteros (1/0)


In [721]:
for columna in df_codificado.columns:
    if df_codificado[columna].dtype == 'bool':
        df_codificado[columna]= df_codificado[columna].astype(int)

eliminamos las columnas de texto que no aportan informacion util al modelo 

In [722]:
df_codificado = df_codificado.drop(columns = ['PassengerId','Name','Ticket'])

mostramoslos tipos de datos 

In [723]:
print (" tipos de datos despues de la correccion")
df_codificado.info()

 tipos de datos despues de la correccion
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Survived        891 non-null    int64  
 1   Pclass          891 non-null    int64  
 2   Age             891 non-null    float64
 3   SibSp           891 non-null    int64  
 4   Parch           891 non-null    int64  
 5   Fare            891 non-null    float64
 6   tiene_Camarote  891 non-null    int64  
 7   Tamaño_Familia  891 non-null    int64  
 8   Sex_male        891 non-null    int64  
 9   Embarked_Q      891 non-null    int64  
 10  Embarked_S      891 non-null    int64  
dtypes: float64(2), int64(9)
memory usage: 76.7 KB


vista previa del data frame

In [724]:
print("\n Vista previa del dataFrame final:")
df_codificado.head()


 Vista previa del dataFrame final:


,Survived,Pclass,Age,SibSp,Parch,Fare,tiene_Camarote,Tamaño_Familia,Sex_male,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0,2,1,0,1
1,1,1,38.0,1,0,71.2833,1,2,0,0,0
2,1,3,26.0,0,0,7.9250,0,1,0,0,1
3,1,1,35.0,1,0,53.1000,1,2,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,1,0,1


### 2. Escalado de Variables Numéricas: Las columnas como Age y Fare (Tarifa) se miden en escalas muy diferentes.
o	Acción Requerida: Aplica un escalado para estandarizarlas. Usa el StandardScaler que vimos en los ejemplos.


In [725]:
# hacemos una copia para este paso final 
df_escalado = df_codificado.copy()

In [726]:
# lista de columnas numericas (incluyendo las que ya eran numeros y las creadas )

columnas_numericas = ['Age','Fare','Pclass','SibSp','Parch','Tamaño_Familia']



ahora creamos una "instancia" del escalador y le decimos que aprenda de nuestros datos y los transforme

In [727]:
#creamos el objeto escalador

scaler = StandardScaler()

In [728]:
#

df_escalado[columnas_numericas] = scaler.fit_transform(df_escalado[columnas_numericas])

In [729]:
df_escalado.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,tiene_Camarote,Tamaño_Familia,Sex_male,Embarked_Q,Embarked_S
0,0,0.827377,-0.565736,0.432793,-0.473674,-0.502445,0,0.059160,1,0,1
1,1,-1.566107,0.663861,0.432793,-0.473674,0.786845,1,0.059160,0,0,0
2,1,0.827377,-0.258337,-0.474545,-0.473674,-0.488854,0,-0.560975,0,0,1
3,1,-1.566107,0.433312,0.432793,-0.473674,0.420730,1,0.059160,0,0,1
4,0,0.827377,0.433312,-0.474545,-0.473674,-0.486337,0,-0.560975,1,0,1


## Fase 3: Creación de un Pipeline de Preprocesamiento (Relacionado con 3. pipelines_preprocesamiento.ipynb)
Objetivo: Tu trabajo hasta ahora es excelente, pero es manual. Necesitamos un proceso automatizado y profesional que podamos reutilizar.


1.	Construir el Pipeline de Preprocesamiento: Usando Pipeline y ColumnTransformer de Scikit-learn, crea un único objeto preprocessor que ejecute todas las acciones de las Fases 1 y 2 de forma automática (rellenar nulos de Age y Embarked, codificar las variables categóricas, y escalar las numéricas). Este es el paso más avanzado, ¡demuéstranos tu habilidad!

In [730]:
# nuestros datos ORIGINALES

df_para_pipeline = df_raw.copy()

### definir las columnas por tipo 

Primero, le decimos a nuestro "director de orquesta" (ColumnTransformer) qué columnas son de qué tipo para que sepa qué hacer con cada una.

In [731]:
# Lista de columnas numéricas que procesaremos
numeric_features = ['Age', 'Fare', 'SibSp', 'Parch', 'Pclass', 'Tamaño_Familia'] 
# (Asumimos que ya creaste Tamaño_Familia, si no, puedes añadirla ahora)

# Lista de columnas categóricas que procesaremos
categorical_features = ['Embarked', 'Sex']

### Paso 3: Crear el Pipeline para las Columnas Numéricas

Le diremos que para cualquier columna numérica, primero rellene los nulos con la mediana y luego la escale.

In [732]:
# Creamos el transformador para datos numéricos
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Paso 1: Rellenar nulos con la mediana
    ('scaler', StandardScaler())                    # Paso 2: Escalar los datos
])

## Paso 4: Crear el Pipeline para las Columnas Categóricas

Le diremos que para cualquier columna categórica, primero rellene los nulos con la moda y luego aplique One-Hot Encoding

In [733]:
# Creamos el transformador para datos categóricos
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Paso 1: Rellenar nulos con la moda
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))  # Paso 2: Aplicar One-Hot Encoding
])

### Paso 5: Ensamblar Todo con ColumnTransformer (El Director de Orquesta)

In [734]:
# Creamos el preprocesador que une los transformadores numéricos y categóricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough' # Le decimos que las columnas no especificadas (ej. Survived) las deje como están
)

## uso del pipeline 


In [735]:
# Separamos las características (X) de nuestro objetivo (y) del dataframe original
# Vamos a dropear columnas que no son características predictivas
df_final = df_para_pipeline.drop(columns=['Name', 'Ticket', 'Cabin', 'PassengerId'])
# Creamos la columna Tamaño_Familia si no lo hicimos antes
if 'Tamaño_Familia' not in df_final.columns:
    df_final['Tamaño_Familia'] = df_final['SibSp'] + df_final['Parch'] + 1


X = df_final.drop('Survived', axis=1)
y = df_final['Survived']

# Aplicamos TODO el preprocesamiento a nuestros datos con UNA SOLA LÍNEA
X_preparado = preprocessor.fit_transform(X)

# X_preparado es ahora un array de NumPy con todos nuestros datos limpios,
# codificados, escalados y listos para un modelo.
print("Dimensiones de los datos preparados:", X_preparado.shape)

Dimensiones de los datos preparados: (891, 9)
